In [3]:
#This is based on the ComboLoss model by Xu and Xiang, found here: https://github.com/lucasxlu/ComboLoss
#It was originally written to be run on Amazon sagemaker. 
#I made several modifications, including:
#1. Changing the number of classification classes
#2. Addingcode to allow for freezing model layers
#3. Changing the method for loading image files



import boto3
import pandas as pd
import sagemaker
import numpy as np
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
import torch
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, models
from torch import nn, optim, as_tensor
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.nn.init import *
from torchvision import transforms, utils, datasets, models
import s3fs
import copy
import os
import sys
import time
import numpy as np
import pandas as pd
import torch.optim as optim
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import recall_score, precision_score, f1_score, confusion_matrix, accuracy_score
from pytorchcv.model_provider import get_model as ptcv_get_model
#sys.path.append('../')
import losses #Find in ComboLoss Github https://github.com/lucasxlu/ComboLoss
from file_util import mkdirs_if_not_exist ##Find in ComboLoss Github
from cfg import cfg #Find in ComboLoss Github
from losses import CombinedLoss, SmoothHuberLoss #Find in ComboLoss Github
from nets import ComboNet #Find in ComboLoss Github


fs = s3fs.S3FileSystem()


sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [6]:
def read_img3(root, dataframe, transform=None):
    output = []
    for _, row in dataframe.iterrows():
        addr = row[0]
        labels = torch.Tensor([row[1]])  # second column
        class_ = torch.Tensor([row[2]]).long()  # third column
        class_ = class_-1 #Columns ran from 1-10, needed 0-9

        try:
            with fs.open(os.path.join(root, addr)) as f:
                img = Image.open(f).convert('RGB')
        except FileNotFoundError:
            print(f"Skipping file: {addr} (File not found)")
            continue

        if transform is not None:
            img = transform(img)

        output.append([img, labels, class_])  # include class in the output

    return output

In [17]:
#Image augmentation for the training data
transform_train  = transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.GaussianBlur(5),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                              ])
                           
bucket = '' #S3 bucket
train_key = 'train_2022_mod_class.txt' #Filenames and labels
root = '' #Folder with images


traindir =  's3://{}/{}'.format(bucket, train_key) #Get information
traindir2 = pd.read_table(traindir, delimiter = ' ', header = None) #Read it
train_dataset = read_img3(root, traindir2, transform=transform_train) #Then read in images
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True) #And put in dataloader


Skipping file: g_7.jpg (File not found)
Skipping file: model-4493512_1920.jpg (File not found)
Skipping file: brown-4197806_1920.jpg (File not found)
Skipping file: pexels-nishant-aneja-2561432.jpg (File not found)
Skipping file: puvvukonvict-photography-AtpSEe3yoIg-unsplash.jpg (File not found)
Skipping file: guy14.jpg (File not found)
Skipping file: kamal-alkhatib-IETO_Z0BrsE-unsplash.jpg (File not found)
Skipping file: brunette-2478405_1920.jpg (File not found)
Skipping file: dollar-gill-nGWo2KbJERc-unsplash.jpg (File not found)
Skipping file: model-633771_1920.jpg (File not found)
Skipping file: guy10.jpg (File not found)
Skipping file: abhimanyu-patel-o2g-PaaeD_U-unsplash.jpg (File not found)
Skipping file: shifaaz-shamoon-MqLy-G-dBi8-unsplash.jpg (File not found)
Skipping file: mateus-campos-felipe-64sktAOWasI-unsplash.jpg (File not found)
Skipping file: kazi-mizan-PDDfWeh44m8-unsplash.jpg (File not found)
Skipping file: rohan-g-T9ckn3XHHnA-unsplash.jpg (File not found)
Skipping 

In [18]:
val_transforms  = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                          ])

val_key = ''
valdir =  's3://{}/{}'.format(bucket, val_key)
valdir2 = pd.read_table(valdir, delimiter = ' ', header = None)
val_dataset = read_img3(root, valdir2, transform=val_transforms)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False)

Skipping file: freckles-2411973_1920.jpg (File not found)
Skipping file: muhammad-murtaza-ghani-ShxuPMbO1lM-unsplash.jpg (File not found)
Skipping file: george-rinon-rJSfWc6qSI8-unsplash.jpg (File not found)
Skipping file: g_9.jpg (File not found)
Skipping file: gregory-buzdyk-bdbC8RlXSUU-unsplash.jpg (File not found)
Skipping file: pexels-ethan-jones-3222422.jpg (File not found)
Skipping file: business-man-1385050_1920.jpg (File not found)


In [9]:
test_transforms  = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                          ])

test_key = 'test_2022_mod_class.txt'
testdir =  's3://{}/{}'.format(bucket, test_key)
testdir2 = pd.read_table(testdir, delimiter = ' ', header = None)
test_dataset = read_img3(root, testdir2, transform=val_transforms)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

Skipping file: children-82272_1920.jpg (File not found)
Skipping file: man-1488305_1920.jpg (File not found)
Skipping file: 116.jpg (File not found)
Skipping file: young-1761404_1920.jpg (File not found)
Skipping file: montgomery-clift-402456_1920.jpg (File not found)
Skipping file: esther-williams-402543_1920.jpg (File not found)
Skipping file: islander-images-kwHBTNUQIqs-unsplash.jpg (File not found)
Skipping file: ashkan-forouzani-YlAp3Y7VUmA-unsplash.jpg (File not found)
Skipping file: pexels-anna-tarazevich-5119607.jpg (File not found)
Skipping file: pexels-nh-noyon-4250862.jpg (File not found)


In [11]:
#Training function

def train_combinator(model, dataloaders, criterion, optimizer, scheduler, num_epochs, inference=False):
    """
    train combinator
    :param model:
    :param dataloaders:
    :param criterion:
    :param optimizer:
    :param scheduler:
    :param num_epochs:
    :param inference:
    :return:
    """
    #print(model)
    model_name = model.__class__.__name__
    model = model.float()
    device = torch.device('cuda:0' if torch.cuda.is_available() and cfg['use_gpu'] else 'cpu')

    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model)
    model = model.to(device)

    dataset_sizes = {x: dataloaders[x].__len__() for x in ['train', 'val', 'test']}

    if not inference:
        print('Start training %s...' % model_name)
        since = time.time()

        best_model_wts = copy.deepcopy(model.state_dict())
        best_record = {
            'pc': 0.0,
            'epoch': 0
        }

        for epoch in range(num_epochs):
            print('-' * 100)
            print('Epoch {}/{}'.format(epoch, num_epochs - 1))

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    if torch.__version__ <= '1.1.0':
                        scheduler.step()
                    model.train()  # Set model to training mode
                else:
                    model.eval()  # Set model to evaluate mode

                running_loss = 0.0
                epoch_gt = []
                epoch_pred = []

                # Iterate over data.
                # for data in dataloaders[phase]:
                for i, data in enumerate(dataloaders[phase], 0):

                    inputs = data[0].to(device)
                    scores = data[1].squeeze(1).to(device).float()
                    classes = data[2].squeeze(1).to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        regression_output, classification_output = model(inputs)
                        regression_output = regression_output.view(-1)
                        _, predicted = torch.max(classification_output.data, 1)

                        loss = criterion(regression_output, scores, F.softmax(classification_output, 1), predicted,
                                         classes)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    epoch_gt += scores.to('cpu').detach().numpy().ravel().tolist()
                    epoch_pred += regression_output.to('cpu').detach().numpy().ravel().tolist()

                if phase == 'train':
                    if torch.__version__ >= '1.1.0':
                        scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]

                epoch_mae = round(mean_absolute_error(np.array(epoch_gt).flatten(), np.array(epoch_pred).flatten()), 4)
                epoch_rmse = round(
                    np.math.sqrt(mean_squared_error(np.array(epoch_gt).flatten(), np.array(epoch_pred).flatten())), 4)
                epoch_pc = round(np.corrcoef(np.array(epoch_gt).flatten(), np.array(epoch_pred).flatten())[0, 1], 4)

                print('[{}] Loss: {:.4f} MAE: {} RMSE: {} PC: {}'
                      .format(phase, epoch_loss, epoch_mae, epoch_rmse, epoch_pc))

                # deep copy the model
                if phase == 'val' and epoch_pc > best_record['pc']:
                    best_record['pc'] = epoch_pc
                    best_record['epoch'] = epoch
                    best_model_wts = copy.deepcopy(model.state_dict())

                    model.load_state_dict(best_model_wts)
                    mkdirs_if_not_exist('./model')
                    torch.save(model.state_dict(),
                               './model/{0}_best_epoch-{1}.pth'.format(model_name, epoch))

        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        print('+' * 100)
        print('Epoch {} achieves best PC: {:4f}'.format(best_record['epoch'], best_record['pc']))
        print('+' * 100)

        # load best model weights
        model.load_state_dict(best_model_wts)
        mkdirs_if_not_exist('./model')
        torch.save(model.state_dict(), './model/%s.pth' % model_name)
    else:
        print('Start testing %s...' % model_name)
        model.load_state_dict(torch.load(os.path.join('./model/%s.pth' % model_name)))

    model.eval()

    total = 0
    y_pred = []
    y_true = []
    filenames = []

    with torch.no_grad():
        for data in dataloaders['test']:
            images = data['image']
            images = images.to(device)

            filenames += data['filename']
            regression_output, classification_output = model(images)
            probs = F.softmax(classification_output, dim=1)
            cls = torch.from_numpy(np.array([[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]], dtype=np.float).T).to(device)  # for SCUT-FBP*
            # cls = torch.from_numpy(np.array([[1.0, 2.0, 3.0]], dtype=np.float).T).to(device)  # for HotOrNot
            # expectation = torch.matmul(probs, cls.float()).view(-1).view(-1, 1)

            # output = (2 * regression_output + expectation) / 3
            output = regression_output
            total += images.size(0)

            y_pred += output.to("cpu").detach().numpy().tolist()
            y_true += data['score'].detach().numpy().tolist()

    mae = round(mean_absolute_error(np.array(y_true).ravel(), np.array(y_pred).ravel()), 4)
    rmse = round(np.math.sqrt(mean_squared_error(np.array(y_true).ravel(), np.array(y_pred).ravel())), 4)
    pc = round(np.corrcoef(np.array(y_true).ravel(), np.array(y_pred).ravel())[0, 1], 4)

    print('===============The Mean Absolute Error of {0} is {1}===================='.format(model_name, mae))
    print('===============The Root Mean Square Error of {0} is {1}===================='.format(model_name, rmse))
    print('===============The Pearson Correlation of {0} is {1}===================='.format(model_name, pc))

    col = ['filename', 'gt', 'pred']
    df = pd.DataFrame([[filenames[i], y_true[i], y_pred[i][0]] for i in range(len(y_true))],
                      columns=col)
    df.to_excel("./{0}.xlsx".format(model_name), sheet_name='Output', index=False)
    print('Output Excel has been generated~')

In [12]:

def main(model, model_type):
    """
    train model
    :param model:
    :param data_name: SCUT-FBP/HotOrNot/SCUT-FBP5500
    :param model_type: classifier/regressor
    :return:
    """
    xent_weight_list = None
    dataloaders = {'train': train_loader, 'val': val_loader, 'test':test_loader}
    xent_weight_list = [170.7, 73.1, 10.0, 3.2, 1.6, 1.1, 1.0, 2.2, 19.0, 512.0] #This refers to the relative frequency of each class in the training data
        
    if model_type == 'regressor':
        # criterion = nn.MSELoss()
        # criterion = nn.SmoothL1Loss()
        # criterion = nn.L1Loss()
        criterion = SmoothHuberLoss() #Custom regression + classification loss function

        optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.01, momentum=0.9, weight_decay=0.001)
        exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)
        train_regressor(model=model, dataloaders=dataloaders, criterion=criterion, optimizer=optimizer,
                        scheduler=exp_lr_scheduler, num_epochs=cfg['epoch'], inference=False)
    elif model_type == 'classifier':
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.001)
        exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)
        train_classifier(model, criterion, optimizer, scheduler=exp_lr_scheduler, dataloaders=dataloaders,
                         num_epochs=cfg['epoch'], inference=False)
    elif model_type == 'combinator':
        criterion = CombinedLoss(xent_weight=xent_weight_list)
        optimizer = optim.SGD(model.parameters(), lr=0.01,  momentum=0.9, weight_decay=0.001)
        exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)
        train_combinator(model, dataloaders, criterion, optimizer, scheduler=exp_lr_scheduler, num_epochs=cfg['epoch'],
                         inference=False)

In [19]:
if __name__ == '__main__':
    seresnext50 = ptcv_get_model("seresnext50_32x4d", pretrained=True)
    num_ftrs = seresnext50.output.in_features
    seresnext50.output = nn.Linear(num_ftrs, 1)
    #Possibility to freeze percentages of layers
    # Calculate the number of layers to freeze
    #num_layers = len(list(seresnext50.features.children()))
    #num_layers_to_freeze = num_layers // 4

    # Get the layers
    #layers = list(seresnext50.features.children())

    # Freeze the first 50% of layers
    #for i in range(num_layers_to_freeze):
     #   for param in layers[i].parameters():
      #      param.requires_grad = False

    # Ensure remaining layers are trainable
    #for i in range(num_layers_to_freeze, num_layers):
     #   for param in layers[i].parameters():
      #      param.requires_grad = True


    # resnet18 = models.resnet18(pretrained=True)
    # num_ftrs = resnet18.fc.in_features
    # resnet18.fc = nn.Linear(num_ftrs, 1)

    main(ComboNet(num_out=10), 'combinator')

Start training ComboNet...
----------------------------------------------------------------------------------------------------
Epoch 0/499
[train] Loss: 98.4180 MAE: 1.4467 RMSE: 1.932 PC: 0.1674
[val] Loss: 83.0404 MAE: 1.0161 RMSE: 1.2439 PC: 0.5164
----------------------------------------------------------------------------------------------------
Epoch 1/499
[train] Loss: 82.5994 MAE: 1.0201 RMSE: 1.2713 PC: 0.4257
[val] Loss: 92.3691 MAE: 1.1804 RMSE: 1.4424 PC: 0.521
----------------------------------------------------------------------------------------------------
Epoch 2/499
[train] Loss: 76.2130 MAE: 0.8678 RMSE: 1.107 PC: 0.5595
[val] Loss: 76.5167 MAE: 0.8716 RMSE: 1.1101 PC: 0.5696
----------------------------------------------------------------------------------------------------
Epoch 3/499
[train] Loss: 72.0632 MAE: 0.7834 RMSE: 1.0021 PC: 0.655
[val] Loss: 75.9597 MAE: 0.8723 RMSE: 1.1303 PC: 0.6315
---------------------------------------------------------------------

KeyboardInterrupt: 